## Setup & Daten laden

In [ ]:
import sys
import os
import math

# Pfad zum src-Ordner hinzufügen
sys.path.insert(0, os.path.abspath('../src'))

# Imports der imperativen Version
from imperative_version import (
    _header, _table, _fmt_pct, _fmt_num, _fmt_int,
    _apply_filters, _success_overall, _duration_stats, 
    _duration_buckets, _group_metrics, _marital_compare,
    _compare_two_groups, _anova_f_balance, _mean, _variance_population
)
from common_io import load_bank_data

# Daten laden
data = load_bank_data()
current = list(data)

print(f"✓ Datensätze geladen: {len(data)}")
print(f"✓ Imperative Version geladen")

---
## Option 1: Erfolgsquote gesamt

Zeigt die Gesamterfolgsquote der Marketingkampagne.

In [ ]:
print(_header("ERFOLGSQUOTE"))
total, yes_count, quote = _success_overall(current)
rows = [
    ["Total", str(total)],
    ["Yes", str(yes_count)],
    ["Quote", _fmt_pct(quote)]
]
print(_table(["Metric", "Value"], rows, aligns=["<", ">"]))

---
## Option 2: Filter setzen

Filtert die Daten nach housing, loan und balance-Kriterien.

**Anleitung:** Passe die Werte unten an:
- `housing_filter`: `True` für yes, `False` für no, `None` für keinen Filter
- `loan_filter`: `True` für yes, `False` für no, `None` für keinen Filter
- `balance_gt_filter`: Mindestwert für balance oder `None`

In [ ]:
# === PARAMETER ANPASSEN ===
housing_filter = None      # None, True oder False
loan_filter = None         # None, True oder False
balance_gt_filter = None   # None oder z.B. 1000

# Filter anwenden
current = _apply_filters(data, housing_filter, loan_filter, balance_gt_filter)

print(_header("FILTER RESULT"))
print(f"Aktueller Datenbestand: {len(current)} / {len(data)}")
print(f"\nAngewendete Filter:")
print(f"  housing = {housing_filter}")
print(f"  loan = {loan_filter}")
print(f"  balance > {balance_gt_filter}")

---
## Option 3: Transformationen

Berechnet Statistiken für transformierte Balance-Werte:
- `log(balance)` - Logarithmische Transformation
- `balance^2+1` - Quadratische Transformation

In [ ]:
print(_header("TRANSFORMATIONEN"))

logs = []
sq1 = []
for row in current:
    bal = row.get("balance")
    if isinstance(bal, (int, float)):
        b = float(bal)
        sq1.append(b * b + 1.0)
        if b > 0.0:
            logs.append(math.log(b))

def stats_line(name, values):
    mu = _mean(values)
    var = _variance_population(values)
    mn = min(values) if values else None
    mx = max(values) if values else None
    return [name, str(len(values)), _fmt_num(mn), _fmt_num(mx), _fmt_num(mu), _fmt_num(var)]

rows = [
    stats_line("log(balance)", logs),
    stats_line("balance^2+1", sq1)
]
print(_table(["Transform", "n", "min", "max", "mean", "var"], rows, aligns=["<", ">", ">", ">", ">", ">"]))

---
## Option 4a: Duration Analyse

Statistische Auswertung der Gesprächsdauer (duration).

In [ ]:
print(_header("DURATION ANALYSE"))
mn, mx, mu, var = _duration_stats(current)
rows = [[_fmt_int(mn), _fmt_int(mx), _fmt_num(mu), _fmt_num(var)]]
print(_table(["min", "max", "mean", "var"], rows, aligns=[">", ">", ">", ">"]))

## Option 4b: Duration Buckets

Gruppierung der Duration in Intervalle mit Erfolgsquoten.

**Anleitung:** Passe `bucket_size` an (z.B. 60 für 60 Sekunden-Intervalle).

In [ ]:
# === PARAMETER ANPASSEN ===
bucket_size = 60  # Intervallgröße in Sekunden

buckets = _duration_buckets(current, bucket_size)
b_rows = [[label, str(cnt), _fmt_pct(rate)] for (label, cnt, rate) in buckets]

print(_header("DURATION BUCKETS"))
print(_table(["bucket", "count", "success"], b_rows, aligns=["<", ">", ">"]))

---
## Option 5: Group by Education

Gruppiert die Daten nach Bildungsniveau und zeigt Durchschnittswerte.

In [ ]:
print(_header("GROUP BY EDUCATION"))
metrics = _group_metrics(current, "education")
rows = []
for name, cnt, avg_age, avg_bal, rate in metrics:
    rows.append([name or "(blank)", str(cnt), _fmt_num(avg_age, 1), _fmt_num(avg_bal, 2), _fmt_pct(rate)])
print(_table(["education", "count", "avg(age)", "avg(balance)", "success"], rows, aligns=["<", ">", ">", ">", ">"]))

---
## Option 6: Group by Marital

Gruppiert die Daten nach Familienstand (single, married, divorced).

In [ ]:
print(_header("GROUP BY MARITAL"))
metrics = _marital_compare(current)
rows = []
for name, cnt, avg_bal, avg_dur, rate in metrics:
    rows.append([name, str(cnt), _fmt_num(avg_bal, 2), _fmt_num(avg_dur, 1), _fmt_pct(rate)])
print(_table(["marital", "count", "avg(balance)", "avg(duration)", "success"], rows, aligns=["<", ">", ">", ">", ">"]))

---
## Option 7: Vergleich zweier Gruppen

Vergleicht zwei spezifische Gruppen innerhalb eines Feldes.

**Anleitung:**
1. Wähle das Feld: `education`, `marital` oder `job`
2. Wähle zwei Gruppen zum Vergleichen

In [ ]:
# === PARAMETER ANPASSEN ===
field = "education"  # "education", "marital" oder "job"
group_a = "tertiary"  # Erste Gruppe
group_b = "secondary" # Zweite Gruppe

# Verfügbare Gruppen anzeigen
available = sorted({(r.get(field) or "") for r in current})
print(f"Verfügbare Gruppen in '{field}': {', '.join(available)}")
print()

# Vergleich durchführen
print(_header("VERGLEICH ZWEIER GRUPPEN"))
m1, m2 = _compare_two_groups(current, field, group_a, group_b)

def row(m):
    name, cnt, avg_age, avg_bal, avg_dur, rate = m
    return [name or "(blank)", str(cnt), _fmt_num(avg_age, 1), _fmt_num(avg_bal, 2), _fmt_num(avg_dur, 1), _fmt_pct(rate)]

rows = [row(m1), row(m2)]
print(_table([field, "count", "avg(age)", "avg(balance)", "avg(duration)", "success"], rows, aligns=["<", ">", ">", ">", ">", ">"]))

delta = m1[-1] - m2[-1]
sign = "+" if delta >= 0 else ""
print(f"\nΔ Erfolgsquote (A-B): {sign}{delta * 100:0.1f}%")

---
## Option 8: ANOVA-ähnlicher F-Wert

Berechnet einen F-Wert für die Balance-Unterschiede zwischen Gruppen.

**Anleitung:** Wähle das Gruppierungsfeld (`education`, `marital` oder `job`).

In [ ]:
# === PARAMETER ANPASSEN ===
field = "education"  # "education", "marital" oder "job"

print(_header("ANOVA-ÄHNLICHER F-WERT (BALANCE)"))
result = _anova_f_balance(current, field)

if result is None:
    print("Nicht genug Daten für F-Berechnung (mind. 2 Gruppen, ausreichend Beobachtungen).")
else:
    f_value, dfb, dfw = result
    f_text = "inf" if math.isinf(f_value) else f"{f_value:0.3f}"
    print(f"F({dfb}, {dfw}) = {f_text}")
    print()
    if math.isinf(f_value):
        print("Interpretation: Innerhalb-Varianz ist 0; Gruppenmittelwerte unterscheiden sich stark")
        print("oder Werte sind konstant pro Gruppe.")
    elif f_value < 1.5:
        print("Interpretation: Eher geringe Unterschiede der Mittelwerte zwischen Gruppen")
        print("(relativ zur Streuung).")
    elif f_value < 5.0:
        print("Interpretation: Moderate Unterschiede der Mittelwerte zwischen Gruppen.")
    else:
        print("Interpretation: Deutliche Unterschiede der Mittelwerte zwischen Gruppen möglich")
        print("(hoher F-Wert).")

---
## Filter zurücksetzen

Setzt den Datensatz auf die ursprünglichen Daten zurück.

In [ ]:
current = list(data)
print(f"✓ Filter zurückgesetzt: {len(current)} Datensätze aktiv")